In [1]:
# necessary for scraping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time
import random

In [2]:
# scraping url for indeed search : data science job in Vancouver BC , radius 50km 
url = 'https://www.glassdoor.ca/Job/canada-data-science-jobs-SRCH_IL.0,6_IN3_KO7,19.htm'

In [3]:
# set up web browser for scraping
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(  
    options=options
)
driver.get(url)
driver.maximize_window()

In [4]:
# expand listing
more_jobs_button = driver.find_element(By.CSS_SELECTOR , ".JobsList_buttonWrapper__haBp5")
more_jobs_button.click()

In [5]:
# close sign-in pop up
try:
    close_button = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".CloseButton")))
    close_button.click()
except TimeoutException: pass

In [6]:
# click the show more jobs button 15 more times
for count in range(15):
    try:
        more_jobs_button = driver.find_element(By.CSS_SELECTOR , ".JobsList_buttonWrapper__haBp5")
        more_jobs_button.click()
    except NoSuchElementException: pass
    time.sleep(4)

In [7]:
# get all job posting element
job_listings = driver.find_elements(By.CSS_SELECTOR, ".JobCard_jobCardWrapper__0b8Jt")

In [10]:
# create variable to store data in job posting
posting_age = None
job_title = None
company_name = None
location = None
pay = None
job_description = None
# create list to store job posting
jobs = []
limit = 5
for job_listing in job_listings:
    # extract posting age
    try:
        posting_age_element = job_listing.find_element(By.CSS_SELECTOR, ".JobCard_listingAge__KuaxZ")
        posting_age = posting_age_element.text
    except NoSuchElementException: pass
    # click job listing
    job_listing.click()
    # wait for 10 sec for the until full job description loaded
    try: 
        job_description_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,".JobDetails_jobDescription__6VeBn")))
    except TimeoutException: pass
    
    # access job right panel
    job = driver.find_element(By.CSS_SELECTOR, ".JobDetails_jobDetailsContainer__sS1W1")
    # click show more
    try:
        show_more_button = job.find_element(By.CSS_SELECTOR,".JobDetails_showMore__j5Z_h")
        show_more_button.click()
        time.sleep(2)
    except NoSuchElementException: pass
    # extract company name
    try: 
        company_name_element = job.find_element(By.CSS_SELECTOR, ".EmployerProfile_employerName__Xemli")
        company_name = company_name_element.text
    except NoSuchElementException: pass
    # extract location
    try: 
        location_element = job.find_element(By.CSS_SELECTOR,".JobDetails_location__MbnUM")
        location = location_element.text
    except NoSuchElementException: pass
    # extract pay
    try:
        pay_element = job.find_element(By.CSS_SELECTOR,".SalaryEstimate_averageEstimate__xF_7h")
        pay_period_element = job.find_element(By.CSS_SELECTOR,".SalaryEstimate_payPeriod__oBnsD")
        pay = pay_element.text + pay_period_element.text
    except NoSuchElementException: pass
    # extract job title
    try: 
        job_title_element = job.find_element(By.CSS_SELECTOR, ".JobDetails_jobTitle__Rw_gn")
        job_title = job_title_element.text
    except NoSuchElementException: pass
    # extract job description
    try:
        job_description_element = job.find_element(By.CSS_SELECTOR,".JobDetails_jobDescription__6VeBn")
        job_description = job_description_element.text
    except NoSuchElementException: pass
    # create job dictionary to store all data for 1 job posting then add to posting list
    job = {}
    job['posting_age'] = posting_age
    job['job_title'] = job_title
    job['company_name'] = company_name
    job['location'] = location
    job['pay'] = pay
    job['job_description'] = job_description
    #print(job)
    jobs.append(job)
    #limit -= 1
    #if limit == 0: break

In [11]:
# quite Chrome driver
driver.quit()

In [16]:
job_df = pd.DataFrame(jobs)
# export data table to csv file
job_df.to_csv(r"C:\Users\bosst\Data Science Job Market Research\data\glassdoor_jobs.csv",index = False)
job_df

,posting_age,job_title,company_name,location,pay,job_description
0,30d+,Développeur Python (Machine Learning/NLP/LLM) ...,SIA Innovations Inc.,Montreal,"$83,378/yr (est.)",Seulement les candidats qui vivent à Montréal ...
1,16d,Junior Data Scientist,Theory and Practice,Vancouver,"$111,030/yr (est.)",Description\nThe Junior Data Scientist will jo...
2,24h,"Director of Algorithms, Modelling & Statistica...",Vancouver Highlanders Rugby,Vancouver,"$75,000/yr (est.)",VANCOUVER HIGHLANDERSRUGBY IS LOOKING FOR THEI...
3,21d,Data Science Intern - Montreal,"Analysis Group, Inc.",Montreal,"$89,644/yr (est.)",Overview:\nAnalysis Group is one of the larges...
4,30d+,Data Science Intern- Summer 2024 (Canada),Altair Engineering,Toronto,"$117,983/yr (est.)",Transforming the Future with the Convergence o...
...,...,...,...,...,...,...
505,7d,DECISION SUPPORT SPECIALIST,St. Mary’s General Hospital,Kitchener,$50.41/hr (est.),Position Description: Temporary Full Time Hybr...
506,30d+,Statistical Science Associate Director/Senior ...,AstraZeneca,Mississauga,$50.41/hr (est.),"At AstraZeneca, we pride ourselves on crafting..."
507,30d+,AI and Machine Learning Principal,Apply Digital,Toronto,"$122,034/yr (est.)",Who we are: We’re a global digital transformat...
508,30d+,Senior Machine Learning Engineer (Remote),OpenTable,Vancouver,"$114,896/yr (est.)","With millions of diners, tens of thousands of ..."
